In [ ]:
# for colab
from google.colab import drive
drive.mount('/content/drive/')

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import shutil

In [2]:
!rm -r ./dataset

rm: cannot remove './dataset': No such file or directory


In [3]:
# only for google colab
# unzip dataset from google drive
!unzip /content/drive/MyDrive/Practicum_Autonomous/track1data.zip -d dataset

unzip:  cannot find or open /content/drive/MyDrive/Practicum_Autonomous/track1data.zip, /content/drive/MyDrive/Practicum_Autonomous/track1data.zip.zip or /content/drive/MyDrive/Practicum_Autonomous/track1data.zip.ZIP.


In [ ]:
# only for kaggle
# copy from track1data from Data folder to /kaggle/working path
src = "/kaggle/input/selfdriving-car-simulator/track1data/track1data"
dest = "./dataset"

shutil.copytree(src, dest)

In [ ]:
# path_csv = "/content/dataset/track1data/track1data/driving_log.csv"
# path_img = "/content/dataset/track1data/track1data/IMG/"

# kaggle
path_csv = "/kaggle/working/dataset/driving_log.csv"
path_img = "/kaggle/working/dataset/IMG/"

In [ ]:
train_df = pd.read_csv(path_csv,
                       names=["center_cam", "left_cam", "right_cam", "steering_angle", "throttle", "reverse", "speed"])
train_df.head()

In [ ]:
len(train_df)

In [ ]:
train_df["center_cam"] = train_df["center_cam"].apply(lambda x: x.split("\\")[-1])
train_df["left_cam"] = train_df["left_cam"].apply(lambda x: x.split("\\")[-1])
train_df["right_cam"] = train_df["right_cam"].apply(lambda x: x.split("\\")[-1])
train_df.head()

In [ ]:
train_df["center_cam"] = path_img + train_df["center_cam"]
train_df["left_cam"] = path_img + train_df["left_cam"]
train_df["right_cam"] = path_img + train_df["right_cam"]
train_df['center_cam'].head()

In [ ]:
plt.hist(train_df.steering_angle.values, bins=20);

In [ ]:
train_df.steering_angle.describe()

In [ ]:
def augment_and_save(img_dir, img_names, label):
    new_imgs = []
    for img_name in img_names:
        img_path = img_name
        img = plt.imread(img_path)
        new_img_name = img_name.replace(".jpg", "") + "_flipped.jpg"
        new_img_path = os.path.join(img_dir, new_img_name)
        cv2.imwrite(new_img_path, cv2.flip(img, 1))
        new_imgs.append((new_img_name, (-label)))
    return new_imgs

In [ ]:
non_zero_df = train_df[train_df["steering_angle"] != 0.0]
non_zero_df

In [ ]:
images_center = []
images_right = []
images_left = []
labels = []

img_dir = path_img
new_imgs = []
for index, data in non_zero_df.iterrows():
    new_images = augment_and_save(img_dir, [data["center_cam"], data["right_cam"], data["left_cam"]], data["steering_angle"])
    images_center.append(new_images[0][0])
    images_right.append(new_images[1][0])
    images_left.append(new_images[2][0])
    labels.append(new_images[0][1])

In [ ]:
augmented_df = pd.DataFrame(list(zip(images_center,images_right, images_left, labels)), columns=["center_cam", "left_cam","right_cam","steering_angle"])

In [ ]:
augmented_df

In [ ]:
zero_df = train_df.query("steering_angle == 0.0").sample(frac=.1)
len(zero_df)

In [ ]:
new_train_df = pd.concat([zero_df, augmented_df, non_zero_df])
new_train_df.head()

In [ ]:
plt.ylim(0, 300)
plt.hist(new_train_df.steering_angle.values, bins=50);

In [ ]:
new_train_df.steering_angle.describe()

In [ ]:
# N=200
# new_train_df.drop(new_train_df[new_train_df['steering_angle'] < 0.10].head(N).index, inplace=True)
# new_train_df.steering_angle.describe()

In [ ]:
# plt.ylim(0, 300)
# plt.hist(new_train_df.steering_angle.values, bins=50);

In [ ]:
new_train_df.to_csv(path_csv, index=False, header=False)

In [ ]:
# every record has 3 images -> available images: 3*len(new_train_df)
len(new_train_df)

### Using Resnet50 
https://arxiv.org/pdf/1912.05440.pdf -> also: worth checking the **3.1**

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F

class TunedResnet50(nn.Module):
    """
    * @brief Initializes the class varaibles
    * @param None.
    * @return None.
    """
    def __init__(self):
        super().__init__()
        self.resnet50 = models.resnet50(weights="IMAGENET1K_V1")
        self.resnet50.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(2048,512),
            nn.ELU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.ELU(),
            nn.Dropout(p=0.5),
            nn.Linear(256, 64),
            nn.ELU(),
            nn.Dropout(p=0.5),
            nn.Linear(64, 1),
            nn.ELU(),
        )
    """ 
    * @brief Function to build the model.
    * @parma The image to train.
    * @return The trained prediction network.
    """
    def forward(self, input):
        input = self.resnet50(input)
        return input
    
    def get_fc_layers(self,):
        return self.resnet50.fc.parameters()
    
    def get_main_layers(self,):
        return [param for name, param in self.resnet50.named_parameters() if 'fc' not in name]

### Nvidia -> https://arxiv.org/pdf/1604.07316.pdf

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F

class NvidiaModel(nn.Module):
    """
    * @brief Initializes the class varaibles
    * @param None.
    * @return None.
    """
    def __init__(self):
        super().__init__()
        self.nvidia = nn.Sequential(
            nn.Conv2d(3, 24, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(24, 36, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(36, 48, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(48, 64, 3),
            nn.ELU(),
            nn.Conv2d(64, 64, 3),
            nn.Dropout(0.5),
            nn.Flatten(),
            nn.Linear(1152, 100),
            nn.ELU(),
            nn.Linear(100, 50),
            nn.ELU(),
            nn.Linear(50, 10),
            nn.ELU(),
            nn.Linear(10, 1),
        )
    """ 
    * @brief Function to build the model.
    * @parma The image to train.
    * @return The trained prediction network.
    """
    def forward(self, input):
        input = self.nvidia(input)
        return input

In [ ]:
!pip install torchsummary

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from torchsummary import summary
 
model = TunedResnet50()
# model = NvidiaModel()

model = model.to(device)

summary(model, (3, 224, 224))

In [ ]:
import csv
import os
from torch.utils.data import Dataset
import cv2


"""
 * @brief The class Features is used to read and seperate various parameters from the 
 * csv file and return them as arrays for the train process.
 * @ param Gets the dataset directory path.
 """
class Features(Dataset):
    """
    * @brief Initializes the parameters in the Features class.
    * @param Path to the dataset.
    * @return None. 
    """
    def __init__(self, path_to_csv):
        #imports the dataset path and joins it to the csv log file.
        path_to_csv = os.path.join(path_to_csv, 'driving_log.csv')
        self.csv_data = self.load_csv_file(path_to_csv)
    """
    * @brief Function to load the csv file and seperate the values written in them.
    * @param Path to the csv file.
    * @return The split and extracted consolidated data.
    """
    def load_csv_file(self, path_to_csv):
        data = []
        with open(path_to_csv, 'r') as csvfile:
            data_reader = csv.reader(csvfile, delimiter=',')
            for row in data_reader:
                data.append(row)
        return data
    """
    * @brief Function to test the return of csv data
    * @param None.
    * @return The csv data
    """ 
    def get_csv_data(self):
        return self.csv_data
    """
    * @brief Function to return the length of the dataset.
    * @param None.
    * @return The length of the dataset.
    """
    def __len__(self):
        return len(self.csv_data)
    """
    * @brief Function to return one data point from the csv file.
    * @param The index of the data to return.
    * @return The data value of the index
    """ 
    def __getitem__(self,i):
        data_entry = self.csv_data[i]
        # Splitting the data of the features from one single data point.
        to_return = {
            'img_center_pth': data_entry[0],
            'img_left_pth': data_entry[1],
            'img_right_pth': data_entry[2],
            'steering_angle': data_entry[3],
            'throttle': data_entry[4],
            'brake': data_entry[5],
            'speed': data_entry[6]
        }

        return to_return

In [ ]:
!pip install tensorboard_logger

In [ ]:
import cv2, os
import numpy as np
import matplotlib.image as mpimg

# declaration of image parameters
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 66, 200, 3
INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

"""
* @brief Function to load the images from the path 
* @param data direcrtory 
* @param File path of the images
* @return The image file
"""
def load_image(data_dir, image_file):
    """
    Load RGB images from a file
    """
    return mpimg.imread(os.path.join(data_dir, image_file.strip()))

"""
* @brief Function to crop the imeages to the required shape
* @param The image to crop
* @return The cropped image
"""
def crop(image):
   
   # Crop the image (removing the sky at the top and the car front at the bottom)
    
    return image[60:-25, :, :] # remove the sky and the car front

"""
* @brief Resize the image to the input shape used by the network model
* @param Image file to Resize
* @return The Resized image for the network
""" 
def resize(image):
    return cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)

"""
* @brief Fuinction to convert the color space of the image from RGB to YUV.
* @param The image to change the colorspace.
* @return The converted image. 
"""
def rgb2yuv(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2YUV)

"""
* @brief Function to run the preprocess of the images.
* @param The images from the dataset
* @return The preprocessed images.
"""
def preprocess(image, train=True):
    image = crop(image)
    image = resize(image)
    if train:
        image = cv2.GaussianBlur(image, (3,3), 0)
    image = rgb2yuv(image)
    return image

"""
* @brief Function to randomly choose the center, left and right images 
* to adjust the steering angles.
* @param Steering angles that corresponds to the images.
* @return The new adjusted steering angles.
"""
def choose_image(steering_angle):
    choice = np.random.choice(3)
    if choice == 0:
        return "img_left_pth", float(steering_angle) + 0.2
    elif choice == 1:
        return "img_right_pth", float(steering_angle) - 0.2
    return "img_center_pth", float(steering_angle)

"""
* @brief Function to randomly flip the left and right images and adjust
* the steering angles.
* @param The images from the left or right dataset.
* @param The steering angle of the corresponding images.
* @return the flipped images and their corresponding steering angles.
"""
def random_flip(image, steering_angle):
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
        steering_angle = -steering_angle
    return image, steering_angle

"""
* @brief Fuinction to randomly translate the image vertically and horizontally.
* @param The image to translate.
* @param The steering angle of the selected image.
* @param Range of x translation.
* @param Range of y translation.
* @return The image and its corresponding steering angle
"""
def random_translate(image, steering_angle, range_x, range_y):
    trans_x = range_x * (np.random.rand() - 0.5)
    trans_y = range_y * (np.random.rand() - 0.5)
    steering_angle += trans_x * 0.002
    trans_m = np.float32([[1, 0, trans_x], [0, 1, trans_y]])
    height, width = image.shape[:2]
    image = cv2.warpAffine(image, trans_m, (width, height))
    return image, steering_angle

"""
* @brief Randomly generates shadows in the image.
* @param The image to add shadow on.
* @return The image with added shadows.
""" 
def random_shadow(image):
    # (x1, y1) and (x2, y2) forms a line
    # xm, ym gives all the locations of the image
    x1, y1 = IMAGE_WIDTH * np.random.rand(), 0
    x2, y2 = IMAGE_WIDTH * np.random.rand(), IMAGE_HEIGHT
    xm, ym = np.mgrid[0:IMAGE_HEIGHT, 0:IMAGE_WIDTH]

    mask = np.zeros_like(image[:, :, 1])
    mask[np.where((ym - y1) * (x2 - x1) - (y2 - y1) * (xm - x1) > 0)] = 1

    # choose which side should have shadow and adjust saturation
    cond = mask == np.random.randint(2)
    s_ratio = np.random.uniform(low=0.2, high=0.5)

    # adjust Saturation in HLS(Hue, Light, Saturation)
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    hls[:, :, 1][cond] = hls[:, :, 1][cond] * s_ratio
    return cv2.cvtColor(hls, cv2.COLOR_HLS2RGB)

"""
* @brief Function to adjust the brightness of the images.
* @param The image to process
* @return The brightness equalized image.
"""
def random_brightness(image):
    # HSV (Hue, Saturation, Value) is also called HSB ('B' for Brightness).
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    ratio = 1.0 + 0.4 * (np.random.rand() - 0.5)
    hsv[:,:,2] =  hsv[:,:,2] * ratio
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

"""
* @brief Function to augment the image and trhe steering angle of the center image
* @param The data directory containing thje images.
* @param The center, left and right images and their corresponding steering angles.
* @param The x and y range of the image augmentation allowd(set to a default value
* unless specified on execution)
* @return Returns the output from all image augmentation process.
"""
def augument(data_dir, center, left, right, steering_angle, range_x=100, range_y=10):
    image, steering_angle = choose_image(data_dir, center, left, right, steering_angle)
    image, steering_angle = random_flip(image, steering_angle)
    image, steering_angle = random_translate(image, steering_angle, range_x, range_y)
    image = random_shadow(image)
    image = random_brightness(image)
    return image, steering_angle

"""
* @brief Function to Generate training image give image paths and associated steering angles
* @param The directory path where the data is stored.
* @param The image paths for each image.
* @param The steering angles for the images.
* @param The variable containing the size of the required batch size.
* @param Binary value of the status of the training process.
* @return The images and steering angles.
"""
def batch_generator(data_dir, image_paths, steering_angles, batch_size, is_training):
    images = np.empty([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])
    steers = np.empty(batch_size)
    while True:
        i = 0
        for index in np.random.permutation(image_paths.shape[0]):
            center, left, right = image_paths[index]
            steering_angle = steering_angles[index]
            # argumentation
            if is_training and np.random.rand() < 0.6:
                image, steering_angle = augument(data_dir, center, left, right, steering_angle)
            else:
                image = load_image(data_dir, center)
            # add the image and steering angle to the batch
            images[i] = preprocess(image)
            steers[i] = steering_angle
            i += 1
            if i == batch_size:
                break
        yield images, steers

In [ ]:
"""
  *  @copyright (c) 2020 Charan Karthikeyan P V, Nagireddi Jagadesh Nischal
  *  @file    train.py
  *  @author  Charan Karthikeyan P V, Nagireddi Jagadesh Nischal
  *
  *  @brief Main file to train and evaluate the model.  
 """
import os
import time
import itertools
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
tqdm.monitor_interval = 0
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tensorboard_logger import configure, log_value
from torch.utils.data.sampler import RandomSampler, SequentialSampler
import matplotlib.pyplot as plt
import argparse
from matplotlib import pyplot as plt
import cv2

history_train_loss = []
history_val_loss = []
"""
* @brief Function to train the model with the input data and save them.
* @param The arguments containing the parameters 
*  needed to train and generate the model.
* @param The model to train the data with.
* @param The split datatset to train.
* @param The validation dataset.
* @return None.
"""
def train_model(args, model, dataset_train, dataset_val):
    # Imports the training model.
    model.train()
    #Declaration of the optimizer and the loss model.
    # used for TunedResNet50
    optimizer = optim.Adam([{'params': model.get_main_layers(), 'lr': 0.00001}, {"params": model.get_fc_layers()}],
                          lr=args['learning_rate'],
                          weight_decay=0.0003)
#     optimizer = optim.Adam(model.parameters(), lr=args['learning_rate'])
    criterion = nn.MSELoss()
    imgs_per_batch = args['batch_size'] #gets the batch size from the argument parameters
    optimizer.zero_grad()
    for epoch in range(args['nb_epochs']): # runs for the number of eposchs set in the arguments
        sampler = RandomSampler(dataset_train)
        train_loss = 0.0
        batch_images = 0 
        for i, sample_id in enumerate(sampler):
            data = dataset_train[sample_id]
#             print("data: ", data)
            label = data['steering_angle'] #, data['brake'], data['speed'], data['throttle']
            # right, center, left images
            for j in range(1,4):
                img_pth, label = choose_image(label)
                # Data augmentation and processing steps
        #             print("img_pth: ", data[img_pth], " label: ", label)
                img = cv2.imread(data[img_pth])

                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = preprocess(img)
                img, label = random_translate(img, label, 100, 10)
                img = random_shadow(img)
                img = random_brightness(img)
                img = Variable(torch.cuda.FloatTensor([img]))
                label = np.array([label]).astype(float)
                label = Variable(torch.cuda.FloatTensor(label))
                img = img.permute(0,3,1,2)
                img = img/255

                #training and loss calculation
                out_vec = model(img)
                loss = criterion(torch.squeeze(out_vec, dim=0),label)
                train_loss += loss.item()

                loss.backward()
                batch_images +=1
                if batch_images%imgs_per_batch==0:
                    optimizer.step()
                    optimizer.zero_grad()
            
        # Validation of the model mid training for better understanding and visualization
        val_loss = eval_model(model,dataset_val)
#                 log_value('val_loss',val_loss,step)
        log_str = \
            'Epoch: {} | Train Loss: {:.8f} | Val Loss: {:.8f}'
        log_str = log_str.format(
            epoch+1,
            train_loss/len(dataset_train),
            val_loss)
        print(log_str)
        model.train()  # resumes the training process
        
        history_val_loss.append(val_loss)
        history_train_loss.append(train_loss/len(dataset_train))

        if (epoch+1)%5==0:
            # Saves the intermediate points in the training process for testing in simulator.
            if not os.path.exists(args['model_dir']):
                os.makedirs(args['model_dir'])

            reflex_pth = os.path.join(
                args['model_dir'],
                'full_resnet_epoch{}.pth'.format(epoch+1))
            torch.save(
                model.state_dict(),
                reflex_pth)

"""
* @brief Function to evaluate the model generated by the training process
* @param Model to be evaluated
* @param The validation dataset
* @param the sample size to evaluate.
* @return The validarion loss.
""" 
def eval_model(model,dataset):
    model.eval()
    criterion = nn.MSELoss()
    val_loss = 0
    sampler = RandomSampler(dataset)
    torch.manual_seed(0)
    for i, sample_id in enumerate(sampler):
        data = dataset[sample_id]
        for j in range(1,4):
            img_pth, label = choose_image(data['steering_angle'])
            # image preprocessing and augmentation.
            img = cv2.imread(data[img_pth])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = preprocess(img, train=False)
    #         img, label = random_flip(img, label)
    #         img, label = random_translate(img, label, 100, 10)
    #         img = random_shadow(img)
    #         img = random_brightness(img)
            img = Variable(torch.cuda.FloatTensor([img]))
            img = img.permute(0,3,1,2)
            label = np.array([label]).astype(float)
            label = Variable(torch.cuda.FloatTensor(label))
            img = img / 255
            out_vec = model(img)
    #         print(out_vec.shape, " ", label.shape)
            loss = criterion(torch.squeeze(out_vec, dim=0),label)

            val_loss += loss.data.item()

    val_loss = val_loss / len(dataset)
    return val_loss

In [ ]:
def main(args):
	#build and import the network model.
    # model = model_cnn()
    model = TunedResnet50()
#     model = NvidiaModel()
#     model = nn.DataParallel(model)
    #Check for cuda availability
    if torch.cuda.is_available():
        model = model.cuda()


    print('Creating model ...')
#     configure("log/")
    print('Creating data loaders ...')
    dataset = Features(args['data_dir'])
    train_size = int(args['train_size'] * len(dataset))
    test_size = len(dataset) - train_size
    print("train size: ", 3*train_size)
    print("test size: ", 3*test_size)
    dataset_train, dataset_val = torch.utils.data.dataset.random_split(dataset,[train_size, test_size])
    
    train_model(args, model,dataset_train, dataset_val)

"""
* @brief Runs the main function and gets the arguments from the user or 
* takes in the default set values
"""
if __name__ == '__main__':
    
    args = {
        'data_dir': '/kaggle/working/dataset', # data path folder
        'model_dir': '/kaggle/working', # where to save the models checkpoints
        'train_size': 0.8,
        'keep_prob': 0.5, # not working
        'nb_epochs': 40,
        'samples_per_epoch': 20000, # not working
        'batch_size': 64,
        'learning_rate': 0.001,
    }
    main(args)

In [ ]:
plt.plot(list(range(1,41)),history_train_loss, label="train_loss")
plt.plot(list(range(1,41)),history_val_loss, label="val_loss")
#     plt.savefig('train_loss(%d,%f,%f).png'%(args['nb_epochs'],args['learning_rate'],args['keep_prob']))
#     plt.clf()
plt.legend()
plt.show()

In [ ]:
# img_path = os.path.join(img_dir, images_center[0])
# img = plt.imread(img_path)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img = preprocess(img)
# label = labels[0]
# img, label = random_flip(img, label)
# img, label = random_translate(img, label, 100, 10)
# img = random_shadow(img)
# img = random_brightness(img)
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))